## Q-1 16 Qubit
Implementations of 16 Qubit Random Number Generator

In [ ]:
# Note: First Create Envirenment Using Vs Code 

# 1) by Powershell as 
     #-python -m venv myvenv
     #-to activate : myvenv/Scripts/activate
# 2) by conda
     #-conda create -p venv python==3.11.0 -y
     #-To activate: conda activate venv/

! pip install qiskit
! pip install qiskit-aer
! pip install pylatexenc

from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit_aer import Aer
from qiskit import transpile

# Create a quantum circuit with 16 qubits
circuit = QuantumCircuit(16, 16)

# Apply Hadamard gates to put all qubits in superposition
circuit.h(range(16))

# Measure all qubits
circuit.measure(range(16), range(16))

# Visualize the circuit
print(circuit)
circuit_drawer(circuit, output='mpl')


# Simulate the quantum circuit using the QASM simulator
simulator = Aer.get_backend('qasm_simulator')

# Use transpile and run
new_circuit = transpile(circuit, simulator)
job = simulator.run(new_circuit, shots=1)
result = job.result()
counts = result.get_counts(circuit)

# Extract the random number from the measurement outcome
random_number = int(list(counts.keys())[0], 2)

# Convert the random number to binary representation
binary_number = bin(random_number)[2:].zfill(16)

print("Random number (decimal):", random_number)
print("Random number (binary):", binary_number)

## Q-2 Error Correction
Tackle Noise with Error Correction 

In [ ]:
! pip install qiskit qiskit-aer
! pip install mthree

# Import necessary modules from Qiskit and mthree
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import mthree

# Define the quantum circuit
qc = QuantumCircuit(3, 3)

# Apply gates and operations to the circuit
qc.h(0)        # Hadamard on qubit 0
qc.cx(0, 1)    # CNOT from qubit 0 to 1
qc.cx(0, 2)    # CNOT from qubit 0 to 2
qc.measure([0, 1, 2], [0, 1, 2])  # Measure all qubits

# Transpile the circuit for the backend
backend = Aer.get_backend('qasm_simulator')
transpiled_qc = transpile(qc, backend)

# Simulate the circuit directly using BackendV2 (no need for assemble)
job = backend.run(transpiled_qc, shots=1000)
result = job.result()
counts = result.get_counts()

# Print original (noisy) measurement results
print(f"Original Counts: {counts}")

# Initialize mthree mitigator and calibrate for qubits [0, 1, 2]
mitigator = mthree.M3Mitigation(backend)
mitigator.cals_from_system([0, 1, 2])

# Apply measurement error mitigation
mitigated_counts = mitigator.apply_correction(counts, qubits=[0, 1, 2]).nearest_probability_distribution()

# Print mitigated counts
print(f"Mitigated Counts: {mitigated_counts}")

# Plot histograms for original and mitigated counts
plot_histogram([counts, mitigated_counts], legend=['Original', 'Mitigated'])

## Q-3 Quantum Teleportation
Implement Quantum Teleportation algorithm in Python.

In [ ]:
!pip install qiskit
!pip install qiskit-aer
!pip install pylatexenc

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')

# Create the quantum circuit with 3 qubits and 3 classical bits
q = QuantumRegister(3, 'q')  # Quantum register
c0 = ClassicalRegister(1, 'c0')  # Classical register for Alice's qubit
c1 = ClassicalRegister(1, 'c1')  # Classical register for Bob's qubit
c2 = ClassicalRegister(1, 'c2')  # Classical register for the result
circuit = QuantumCircuit(q, c0, c1, c2)

# Prepare the initial state to be teleported
circuit.initialize([0, 1], q[0])  # Initialize q[0] in the |1> state
circuit.barrier()

# Create an entanglement between Alice's and Bob's qubits
circuit.h(q[1])  # Hadamard on Bob's qubit
circuit.cx(q[1], q[2])  # CNOT between Bob's qubits
circuit.barrier()

# Teleportation process
circuit.cx(q[0], q[1])  # CNOT for teleportation
circuit.h(q[0])  # Hadamard on Alice's qubit
circuit.barrier()

# Measure Alice's qubits and send the measurement results to Bob
circuit.measure(q[0], c0[0])  # Measure Alice's qubit
circuit.measure(q[1], c1[0])  # Measure Bob's qubit

# Apply corrective operations on Bob's qubit based on the measurement results
circuit.x(q[2]).c_if(c1, 1)  # Apply X gate if Bob's measurement is 1
circuit.z(q[2]).c_if(c0, 1)  # Apply Z gate if Alice's measurement is 1

# Measure the teleported qubit
circuit.measure(q[2], c2[0])  # Measure the teleported state

# Visualize the circuit
print(circuit)
circuit.draw(output='mpl')

# Simulate the circuit using the QASM simulator
transpiled_circuit = transpile(circuit, backend)  # Transpile the circuit for the simulator
job = backend.run(transpiled_circuit, shots=1)  # Run the transpiled circuit
result = job.result()
teleported_state = result.get_counts(transpiled_circuit)  # Get counts from the transpiled circuit

# Print the teleported state
print("Teleported state:", teleported_state)

## Q-4 Randomized Benchmark
The Randomized Benchmarking Protocol.

In [ ]:
! pip install qiskit
! pip install qiskit-aer

import numpy as np
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit_aer import Aer
from qiskit.quantum_info import Statevector, state_fidelity


# Generate a random quantum circuit
def generate_random_circuit(num_qubits, depth):
    circuit = QuantumCircuit(num_qubits)
    for _ in range(depth):
        for qubit in range(num_qubits):
            circuit.rx(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.ry(np.random.uniform(0, 2 * np.pi), qubit)
            circuit.rz(np.random.uniform(0, 2 * np.pi), qubit)
        for qubit in range(num_qubits - 1):
            circuit.cz(qubit, qubit + 1)
    return circuit


# Perform randomized benchmarking
def randomized_benchmarking(num_qubits, depths, num_sequences, shots):
    backend = Aer.get_backend('aer_simulator')  # AerSimulator backend
    results = []
    for depth in depths:
        success_counts = 0
        for _ in range(num_sequences):
            # Generate a random circuit and its inverse
            circuit = generate_random_circuit(num_qubits, depth)
            inverse_circuit = circuit.inverse()

            # Transpile the circuits for the backend
            transpiled_circuit = transpile(circuit, backend)
            transpiled_inverse_circuit = transpile(inverse_circuit, backend)

            # Apply the circuit and obtain the final statevector
            transpiled_circuit.save_statevector()  # Ensure statevector is saved
            circuit_result = backend.run(transpiled_circuit).result()  # Run the transpiled circuit
            final_statevector = Statevector(circuit_result.get_statevector())

            # Apply the inverse circuit and obtain the final statevector
            transpiled_inverse_circuit.save_statevector()  # Ensure statevector is saved
            inverse_result = backend.run(transpiled_inverse_circuit).result()  # Run the transpiled inverse circuit
            inverse_statevector = Statevector(inverse_result.get_statevector())

            # Calculate the fidelity
            fidelity = state_fidelity(final_statevector, inverse_statevector)
            success_counts += shots * (1 - fidelity)

        success_rate = success_counts / (num_sequences * shots)
        results.append(success_rate)
    return results

# Example usage
num_qubits = 2
depths = [1, 2, 3, 4]
num_sequences = 100
shots = 1024

results = randomized_benchmarking(num_qubits, depths, num_sequences, shots)
print(results)

## Q-5 5 Qubit
Implementing a 5 qubit Quantum Fourier Transform.

In [ ]:
! pip install qiskit
! pip install qiskit-aer
! pip install pylatexenc


from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit_aer import Aer  # Corrected import for Aer simulator
from qiskit.circuit.library import QFT
import numpy as np

# Use Aer simulator backend
backend = Aer.get_backend('qasm_simulator')

# Define pi constant
pi = np.pi

# Create Quantum and Classical Registers
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

# Create Quantum Circuit
circuit = QuantumCircuit(q, c)

# Apply some X gates
circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])

# Apply Quantum Fourier Transform (QFT)
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q)

# Measure the quantum register
circuit.measure(q, c)

# Draw the circuit
circuit.draw(output='mpl', filename='qft1.png')

print(circuit)

# Transpile the circuit for the Aer backend
transpiled_circuit = transpile(circuit, backend)

# Execute the circuit
job = backend.run(transpiled_circuit, shots=1000)

# Get the results
result = job.result()
counts = result.get_counts()

# Display the QFT output
print("\nQFT Output")
print("-------------")
print(counts)

# Second Circuit: Apply Inverse QFT
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

circuit = QuantumCircuit(q, c)

# Apply the same X gates as before
circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])

# Apply Quantum Fourier Transform (QFT)
circuit.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=True, insert_barriers=False, name='inverse_qft'), q)

# Measure the quantum register
circuit.measure(q, c)

# Draw the second circuit
circuit.draw(output='mpl', filename='qft2.png')

print(circuit)

# Transpile the second circuit for the Aer backend
transpiled_circuit = transpile(circuit, backend)

# Execute the second circuit
job = backend.run(transpiled_circuit, shots=1000)

# Get the results for the second job
result = job.result()
counts = result.get_counts()

# Display the QFT with inverse QFT output
print("\nQFT with inverse QFT Output")
print("------------------------------")
print(counts)